## Descripción del algoritmo


Se ha desarrollado un código en Python 3 a partir de los modelos globales. Se pretende comprobar si los resultados de la simulación son compatibles con datos disponibles de generación de un campo solar real.
Se ha seguido el paradigma de Programación Orientada a Objetos, de tal forma que la planta solar y sus diferentes subsistemas se modelan mediante diferentes Clases. Estas clases disponen de métodos para la entrada/salida de información, así como atributos necesarios para caracterizar cada Objeto.
En la descripción de las clases se seguirá un proceso ascendente en la estructura constructiva del campo solar, comenzando con los objetos más elementales, los HCE, y terminando con el campo solar en su conjunto. Además, otras clases se desarrollan, aunque de una forma muy elemental, para permitir futuras integraciones con otros elementos de la planta (Bloque de Potencia, Tren de Generación, Sistema BOP, etc.) 

### Clase HCE (Heat Collector Element)

Físicamente, un HCE es un tubo de acero con una envolvente de vidrio de tal forma que entre el tubo de acero y la envolvente queda un espacio en el que se ha practicado el vacío. Por el interior del tubo circula el fluido caloportador (HTF, Heat Transfer Fluid) que aumenta su energía térmica al recibir el calor procedente de las paredes interiores del tubo.
El tubo recibe durante el proceso de fabricación, un recubrimiento selectivo que mejora sus propidades físicas para absorber la raciación solar. 
De cara a modelar el funcionamiento del HCE como elemento responsable de calentar el HTF, se define la clase HCE que consta de los siguientes atributos:

* Temperatura de entrada del HTF, tin
* Presión de entrada del HTF, pin
* Caudal másico del HTF, massflow
* Potencia calorífica absorbida, qabs
* Potencia calorífica perdida, qperd
* Rendimiento global del HCE, pr
* Temperatura de salida del HTF, tout
* Presión de salida del HTF, pout


Con estos parámetros el comportamiento del HCE queda totalmente caracterizado en el sistema desde el punto de vista del proceso de generación. No todos estos atributos (pueden entenderse como variables) son independientes entre sí. Por ejemplo, el rendimiento es:

$ pr = 1 - q_{perd} / q_{abs} $

Más adelante veremos como un objeto (instancia) de la clase HCE puede ser procesada por otra instancia de la clase Modelo para modelar su comportamiento según el modelo teórico que se pretende validar.

La Clase HCE también nos proporciona algunos métodos necesarios para el trabajo de procesamiento de la información, asignación y recuperación de valores de los atributos.

Otro asepecto importante es que cada instancia de la clase HCE tiene un atributo de tipo 'diccionario', en el que a modo de lista de pares clave-valor va a recibir aquellos parámetros que posteriormente serán empleados por el Modelo. Los diferentes autores que han elaborado modelos para los HCE no siempren utilizan los mismos parámetros ni idénticos identificadores. Al emplear un diccionario se facilita la tarea de implementación de nuevos Modelos, sin que sea necesario cambiar los atributos de la clase en cada ocasión.

Finalmente, un HCE es un elemento que ocupa una determinada posición dentro del SCA (Solar Collector Assembly). Más adelante se verá que, para determinadas simulaciones, el orden que ocupa dentro de la fila de HCEs y el propio SCA al que pertenece, pueden ser datos necesarios a la hora de estudiar su comportamiento. Por este motivo cada HCE mantiene una referencia al SCA al que pertenece y guarda información sobre su posición relativa dentro de él.






In [ ]:
class HCE(object):

    def __init__(self, sca, hce_order, settings):

        self.sca = sca
        self.hce_order = hce_order
        self.parameters = dict(settings)
        self.tin = 0.0
        self.tout = 0.0
        self.pr = 0.0
        self.qabs = 0.0
        self.qperd = 0.0
        self.pout = 0.0
        self.pin = 0.0

    def set_tin(self):

        if self.hce_order > 0:
            self.tin = self.sca.hces[self.hce_order-1].tout
        elif self.sca.sca_order > 0:
            self.tin = self.sca.loop.scas[self.sca.sca_order-1].hces[-1].tout
        else:
            self.tin = self.sca.loop.tin

    def set_pin(self):

        if self.hce_order > 0:
            self.pin = self.sca.hces[self.hce_order-1].pout
        elif self.sca.sca_order > 0:
            self.pin = self.sca.loop.scas[self.sca.sca_order-1].hces[-1].pout
        else:
            self.pin = self.sca.loop.pin

    def set_tout(self, htf):

        HL = htf.get_deltaH(self.tin, self.sca.loop.pin)

        if self.qabs > 0:

            h = (np.pi * self.parameters['Absorber tube outer diameter'] *
                 self.parameters['Length'] *
                 self.qabs * self.pr / self.sca.loop.massflow)

        else:
            h = -self.qperd / self.sca.loop.massflow

        self.tout = htf.get_T(HL + h, self.sca.loop.pin)


    def set_pout(self, htf):

        # TO-DO CÁLCULLO PERDIDA DE CARGA:
        # Ec. Colebrook-White para el cálculo del factor de fricción de Darcy

        re_turbulent = 4000

        k = self.parameters['Inner surface roughness']
        D = self.parameters['Absorber tube inner diameter']
        re = htf.get_Reynolds(D, self.tin, self.pin, self.sca.loop.massflow)


        if re < re_turbulent:
            darcy_factor = 64 / re

        else:
            # a = (k /  D) / 3.7
            a = k / 3.7
            b = 2.51 / re
            x = 1

            fx = lambda x: x + 2 * np.log10(a + b * x )

            dfx = lambda x: 1 + (2 * b) / (np.log(10) * (a + b * x))

            root = sc.optimize.newton(fx,
                                      x,
                                      fprime=dfx,
                                      maxiter=10000)

            darcy_factor = 1 / (root**2)


        rho = htf.get_density(self.tin, self.pin)
        v = 4 * self.sca.loop.massflow / (rho * np.pi * D**2)
        g = sc.constants.g

        # Ec. Darcy-Weisbach para el cálculo de la pérdida de carga
        deltap_mcl = darcy_factor * (self.parameters['Length'] / D ) * (v**2 / (2 * g))

        deltap = deltap_mcl * rho * g

        self.pout = self.pin - deltap

    def set_krec(self, t):

        self.parameters['krec'] = (0.0153)*(t - 273.15) + 14.77

    def get_previous(self):

        return self.sca.hces[self.hce_order-1]

    def get_index(self):

        if hasattr(self.sca.loop, 'subfield'):
            index = [self.sca.loop.subfield.name,
                self.sca.loop.loop_order,
                self.sca.sca_order,
                self.hce_order]
        else:
            index = ['prototype',
                self.sca.loop.loop_order,
                self.sca.sca_order,
                self.hce_order]

        return index

    def get_pr_opt_peak(self, aoi, solarpos, row):

        alpha = self.get_absorptance()
        tau = self.get_transmittance()
        rho = self.sca.parameters['Reflectance']
        gamma = self.sca.get_solar_fraction(aoi, solarpos, row)

        pr_opt_peak = alpha * tau * rho * gamma

        if pr_opt_peak > 1 or pr_opt_peak < 0:
            print("ERROR", pr_opt_peak)

        return pr_opt_peak


    def get_pr_geo(self, aoi, solarpos, row):

        if aoi > 90:
            pr_geo = 0.0

        else:
            # Llamado "bordes" en Tesis. Pérdidas de los HCE de cabecera según aoi
            sca_unused_length = (self.sca.parameters["Focal Len"] *
                                 np.tan(np.radians(aoi)))

            unused_hces = sca_unused_length // self.parameters["Length"]

            unused_part_of_hce = ((sca_unused_length % self.parameters["Length"]) /
                                  self.parameters["Length"])

            if self.hce_order < unused_hces:
                pr_geo = 0.0

            elif self.hce_order == unused_hces:
                pr_geo = ((sca_unused_length % self.parameters["Length"]) /
                                  self.parameters["Length"])
            else:
                pr_geo = 1.0

            # pr_geo = 1- (self.sca.parameters["Focal Len"] * np.tan(np.radians(aoi)) /
            #              (len(self.sca.hces) * self.parameters["long"]))

            if pr_geo > 1.0 or pr_geo < 0.0:
                print("ERROR pr_geo out of limits", pr_geo)


        return pr_geo


    def get_pr_shadows(self, aoi, solarpos, row):

        # Llamado "sombras" en Tesis. Pérdidas por sombras. ¿modelar sobre el SCA?
        # Sombras debidas a otros lazos

        shadowing = (1 -
                     np.sin(np.radians(abs(solarpos['elevation'][0]))) *
                     self.sca.loop.row_spacing /
                     self.sca.parameters['Aperture'])

        if shadowing < 0.0:
            shadowing = 0.0

        if solarpos['elevation'][0] < 0:
            shadowing = 1.0

        pr_shadows = 1 - shadowing

        if  pr_shadows > 1 or  pr_shadows < 0:
            print("ERROR",  pr_shadows)

        return pr_shadows


    def get_hext(self, wspd):

        #  TO-DO:

        return 0.0


    def get_emittance(self, tro, wspd):


        #  Eq. 5.2 Barbero
        eext = (self.parameters['Absorber emittance factor A0'] +
                self.parameters['Absorber emittance factor A1'] *
                (tro - 273.15))
        """
        Lineal Increase if wind speed lower than 4 m/s up to 1% at 4 m/s
        Lineal increase over 4 m/s up to 2% at 7 m/s
        """
        if wspd <4:
            eext = eext * (1 + 0.01 * wspd / 4)

        else:
            eext = eext * (1 + 0.01 * (0.3333 * wspd - 0.3333))

        return eext


    def get_absorptance(self):

        #
        alpha = self.parameters['Absorber absorptance']

        return alpha


    def get_transmittance(self):

        #dependiendo si hay vídrio y si hay vacío
        tau = self.parameters['Envelope transmittance']

        return tau


    def get_reflectance(self):

        return self.sca.parameters['Reflectance']